In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# import_ipynb module 설치
!pip install import_ipynb

!apt-get update
!apt-get -qq install xxd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 4.0 MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [985 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB

In [3]:
# import를 위한 경로이동
%cd /content/drive/MyDrive/team_malmungchi/colab/speaker_verification/code
!ls

/content/drive/.shortcut-targets-by-id/1rEYIox5uYM9uP2spDttCYCqtt2GvSbn1/team_malmungchi/colab/speaker_verification/code
batcher.ipynb		 constants.ipynb     fitter.ipynb      test_eer.ipynb
call_development1.ipynb  convert.ipynb	     loss.ipynb        utils.ipynb
call_development2.ipynb  development.ipynb   network.ipynb
call_development3.ipynb  eval_metrics.ipynb  preprocess.ipynb
call_tflite.ipynb	 evaluation.ipynb    temp.wav


In [4]:
import tensorflow as tf
from tensorflow import keras
import pickle
import numpy as np

import import_ipynb
import network
from utils import load_best_checkpoint

importing Jupyter notebook from network.ipynb
importing Jupyter notebook from eval_metrics.ipynb
importing Jupyter notebook from utils.ipynb
importing Jupyter notebook from constants.ipynb


In [5]:
def convert(model_name, checkpoint_dir, model_path):
  Model = network.get_network(model_name)

  best_ckpt = load_best_checkpoint(checkpoint_dir)
  Model.load_weights(checkpoint_dir+'/'+best_ckpt)

  converter = tf.lite.TFLiteConverter.from_keras_model(Model)
  converter.optimizations = [tf.lite.Optimize.DEFAULT]
  converter._experimental_default_to_single_batch_in_tensor_list_ops = True # lstm의 경우 배치 크기가 1임을 명시해줘야한다.

  #########################################
  with open('../data/dataset/train_cpp_1581_200_49_40.pickle',"rb") as f:
    X = pickle.load(f)
    X = X.astype(np.float32)

  def representative_data_gen():
    for input_value in X:
      yield [input_value]

  converter.representative_dataset = representative_data_gen

  # Ensure that if any ops can't be quantized, the converter throws an error
  converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
  # Set the input and output tensors to uint8 (APIs added in r2.3)
  converter.inference_input_type = tf.int8
  converter.inference_output_type = tf.int8

  ########################################

  tflite_model = converter.convert()
  open(model_path+"/"+checkpoint_dir.split('/')[-1]+".tflite", "wb").write(tflite_model)

In [6]:
convert('arduino_CRNN','../model/arduino_CRNN-naive_batcher-cross_entropy', '../model')

float32 -128.0


/usr/local/lib/python3.7/dist-packages/tensorflow/lite/python/convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


In [7]:
%cd ..
%cd model
!ls

/content/drive/.shortcut-targets-by-id/1rEYIox5uYM9uP2spDttCYCqtt2GvSbn1/team_malmungchi/colab/speaker_verification
/content/drive/.shortcut-targets-by-id/1rEYIox5uYM9uP2spDttCYCqtt2GvSbn1/team_malmungchi/colab/speaker_verification/model
ACRNN-naive_batcher-cross_entropy
ACRNN-simMat_batcher-simMat_loss
ACRNN-simMat_batcher-simMat_loss--transferedFrom--ACRNN-naive_batcher-cross_entropy
ADRNN-naive_batcher-cross_entropy
ADRNN-simMat_batcher-simMat_loss
ADRNN-simMat_batcher-simMat_loss-931-large--transferedFrom--ADRNN-naive_batcher-cross_entropy
ADRNN-simMat_batcher-simMat_loss-931--transferedFrom--ADRNN-naive_batcher-cross_entropy
ADRNN-simMat_batcher-simMat_loss--transferedFrom--ADRNN-naive_batcher-cross_entropy
arduino_CNN-naive_batcher-cross_entropy
arduino_CNN-naive_batcher-cross_entropy.cc
arduino_CNN-naive_batcher-cross_entropy-noCustom.cc
arduino_CNN-naive_batcher-cross_entropy-noCustom.tflite
arduino_CNN-naive_batcher-cross_entropy.tflite
arduino_CRNN-naive_batcher-cross_entropy

In [8]:
!xxd -i arduino_CRNN-naive_batcher-cross_entropy.tflite > arduino_CRNN-naive_batcher-cross_entropy.cc